In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
training = fc.eightb.training.training_5M.NMSSM_XYY_YToHH_8b_MX_1000_MY_450

In [4]:
def get_train(path):
    return path.replace('ntuple.root','reweighted_ntuple_0.root')

In [5]:
training = Tree( get_train(training) )

100%|██████████| 1/1 [00:03<00:00,  3.03s/it]


In [6]:
trees = ObjIter([training])

In [7]:
trees.apply(lambda t: build_collection(t, 'H\dY\d_b\d', 'jet', ordered='pt'))

[None]

In [8]:
jet_p4 = build_p4(training, 'jet', use_regressed=True)

In [39]:
j1, j2 = ak.unzip(ak.combinations(jet_p4, n=2))

In [40]:
hm = (j1 + j2).m/125

In [41]:
m = hm[0]

In [42]:
i1, i2 = ak.unzip(ak.combinations(np.arange(8),2,axis=0))

In [78]:
def get_m(i, j):
    if i > j: i, j = j, i

    return m[(i1 == i) & (i2 == j)][0]

In [72]:
def distance(p0, p1):
    return np.sqrt( np.sum( (p0-p1)**2 ) )

In [74]:
j0 = np.zeros(8,)

In [79]:
j1 = np.zeros(8,)
j1[0] = get_m(0, 1)

In [80]:
distance(j0,j1)

9.34167766571045

In [93]:
from scipy.optimize import fmin

In [97]:
def pairwise(p, *args):
    ndim = p.shape[0]
    loss = 0
    for pi, di in args:
        loss += (distance(p, pi[:ndim]) - di)**2
    return loss


pairwise(j2[:2], (j0, get_m(2, 0)), (j1, get_m(2, 1)))

81.93596233229641

In [107]:
j0 = np.zeros(8,)

j1 = np.zeros(8,)
j1[:1] = fmin(pairwise, j1[:1], args=((j0, get_m(1, 0)),))

j2 = np.zeros(8,)
j2[:2] = fmin(pairwise, j2[:2], args=((j0, get_m(2, 0)), (j1, get_m(2, 1))))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 31
         Function evaluations: 62
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 78
         Function evaluations: 151


In [111]:
j3= np.zeros(8,)
j3[:3] = fmin(pairwise, j2[:3], args=((j0, get_m(3, 0)), (j1, get_m(3, 1)), (j2, get_m(3, 2))))

Optimization terminated successfully.
         Current function value: 1.149184
         Iterations: 64
         Function evaluations: 117


In [144]:
js = [np.zeros(8,)]
i = 0

In [160]:
i += 1
ji = np.zeros(8,)

ds = tuple([ (p, get_m(i, j)) for j,p in enumerate(js) ])
ji[:i] = fmin(pairwise, ji[:i], args=ds)
js.append(ji)

ValueError: in NumpyArray attempting to get 0, index out of range

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/libawkward/array/NumpyArray.cpp#L1217)

In [161]:
js[-1][:i]

array([ 5.7330928 ,  1.14583616, -0.64531705, -0.15591816, -1.25910414,
       -0.4198025 , -1.47007206,  0.        ])

In [159]:
js

[array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([9.3416875, 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       ]),
 array([6.68497543, 3.11106914, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]),
 array([ 7.44496256e+00,  9.40241147e-02, -3.65616760e-05,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 array([ 6.99741132e+00,  1.74649745e+00, -1.28574411e-03, -1.54579176e-03,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 array([ 5.65115625,  0.69097385, -0.34132619, -1.68968632, -0.63316113,
         0.        ,  0.        ,  0.        ]),
 array([ 6.08635282,  0.68130567, -0.57411256, -1.55869327, -1.33187088,
        -0.30082523,  0.        ,  0.        ]),
 array([ 5.7330928 ,  1.14583616, -0.64531705, -0.15591816, -1.25910414,
        -0.4198025 , -1.47007206,  0.        ])]